In [1]:
from requests_html import HTMLSession, HTML
import json
import pandas
import pathlib
import pydash
import re
import requests
import time
import uuid

links = list()

r = [x for x in range(1958, 2011) if x not in [1961, 1974]]
c = list()
for x in [1969, 1979, 1989, 1999]:
    l = r.index(x)+1
    c.append(r[:l])
    r = r[l:]
c.append(r)

for a in c:
    for b in a:
        links.append(f'https://www.aacta.org/aacta-awards/winners-and-nominees/range/{min(a)}-{max(a)}/year/{b}/')

for x in ['inaugural-aacta-awards']+['2nd', '3rd', '4th', '5th', '6th', '7th']+[str(x) for x in range(2018,2022)]:
    if 'ina' not in x:
        x = f'{x}-aacta-awards'
    links.append(f'https://www.aacta.org/aacta-awards/winners-and-nominees/{x}')

dataframe = pandas.DataFrame(columns=['ceremony', 'page'])

for l in links:
    time.sleep(4)
    session = HTMLSession()
    r = session.get(l)
    for x in r.html.find('.filter-list__button', first=False):
        dataframe.loc[len(dataframe)] = [(pathlib.Path(l).stem), (x.attrs['data-load-winners'])]

dataframe['ceremony'] = dataframe['ceremony'].str.replace('1995', '1995-2')
dataframe['ceremony'] = dataframe['ceremony'].str.replace('1996', '1996-2')
dataframe['ceremony'] = dataframe['ceremony'].str.replace('2006', '2006-2')

dataframe = dataframe.loc[dataframe.page.str.contains('film', na=False)]
dataframe = dataframe.loc[~dataframe.page.str.contains('short', na=False)]
dataframe = dataframe.loc[~dataframe.page.str.contains('non', na=False)]

print(len(dataframe))
dataframe.head()

46


,ceremony,page
61,1976,feature-film
64,1977,feature-film
67,1978,feature-film
70,1979,feature-film
73,1980,feature-film


In [2]:
stream = pandas.DataFrame()
for x in dataframe.to_dict('records'):
    
    if x['ceremony'][:2] in ['19', '20']:
        x['year'] = x['ceremony'][:4]
    elif x['ceremony'] == '1995':
        x['year'] = '1995-2'        
    elif x['ceremony'] == 'inaugural-aacta-awards':
        x['year'] = '2011'
    else:
        x['year'] = '201'+x['ceremony'][:1] 

    save_path = pathlib.Path.cwd() / 'data' / f"{x['year']}-{x['page']}.csv"
    if not save_path.exists():
        time.sleep(4)
        url = f"https://www.aacta.org/aacta-awards/awards-history/{x['ceremony']}/loadwinners/{x['page']}"
        r = requests.get(url)
        if r.status_code == 200:
            data = pydash.get(json.loads(r.content.decode('utf-8')), "winners")   
            data = data.split('awards-list__title heading heading--lv3')[1:]
            for d in data:
                award_data = pandas.DataFrame(columns=['candidate'])        
                html = HTML(html=d)
                award_data['candidate'] = [x.text for x in (html.find('.nominees-list__title'))]
                award_data['info'] = [x.text for x in (html.find('.nominees-list__info'))]
                award_data['winner'] = ['winner' in str(x) for x in (html.find('.nominees-list__item'))]
                award_data['award_name'] = d[2:].split('</h3>')[0]
                award_data['year'] = x['year']
                award_data['page'] = x['page']
                stream = pandas.concat([stream, award_data])
        else:
            raise Exception(url, 'not found.')

print(len(stream))
stream.head()

2570


,candidate,info,winner,award_name,year,page
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film
1,Picnic at Hanging Rock,"Patricia Lovell, Hal McElroy, Jim McElroy",False,Best Film sponsored by the Australian Film Com...,1976,feature-film
0,The Devil's Playground,Fred Schepisi,True,Best Direction sponsored by Village Theatres,1976,feature-film
1,Pure Shit,Bert Deling,False,Best Direction sponsored by Village Theatres,1976,feature-film
2,End Play,Tim Burstall,False,Best Direction sponsored by Village Theatres,1976,feature-film


In [3]:
def string_clean(row):

    ''' Remove text within brackets. '''

    text = row['info']
    text = re.sub(r'\([^)]*\)', '', text)
    return text

best_film = stream.copy() 
best_film = best_film.loc[best_film.award_name.str.contains('Best Film', na=False)]
best_film['info'] = best_film['info'].str.split('\n').str[0]
best_film['info'] = best_film.apply(string_clean, axis=1)

best_film['info'] = best_film['info'].str.split(',')
best_film = best_film.explode('info')
best_film['info'] = best_film['info'].str.strip()

best_film['label'] = best_film['candidate']+' ('+best_film['year']+')'

work_uuid = best_film.copy()
work_uuid = work_uuid[['label']].drop_duplicates()
work_uuid['work_uuid'] = [str(uuid.uuid4()) for x in range(len(work_uuid))]
best_film = pandas.merge(best_film, work_uuid, on='label', how='left')

agent_uuid = best_film.copy()
agent_uuid = agent_uuid[['info']].drop_duplicates()
agent_uuid['agent_uuid'] = [str(uuid.uuid4()) for x in range(len(agent_uuid))]
best_film = pandas.merge(best_film, agent_uuid, on='info', how='left')

best_film.to_csv(pathlib.Path.cwd() / 'aacta.csv', index=False)
print(len(best_film))
best_film.head()

395


,candidate,info,winner,award_name,year,page,label,work_uuid,agent_uuid
0,The Devil's Playground,Fred Schepisi,True,Best Film sponsored by the Australian Film Com...,1976,feature-film,The Devil's Playground (1976),2325ab2f-0ac4-4f59-96e0-3f17faed105a,b80121d1-23bd-41c3-85fc-1cc0264596c5
1,Picnic at Hanging Rock,Patricia Lovell,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,Picnic at Hanging Rock (1976),1c17ae58-10da-45cd-b76c-e9633e170479,c6a6eabc-c4fc-4160-a261-1c6904a14077
2,Picnic at Hanging Rock,Hal McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,Picnic at Hanging Rock (1976),1c17ae58-10da-45cd-b76c-e9633e170479,34395e0d-8e89-4382-b852-920b0134da85
3,Picnic at Hanging Rock,Jim McElroy,False,Best Film sponsored by the Australian Film Com...,1976,feature-film,Picnic at Hanging Rock (1976),1c17ae58-10da-45cd-b76c-e9633e170479,06754f19-313d-4142-a15b-2ed467cbeaa2
4,Break Of Day,Patricia Lovell,False,Best Film sponsored by the Australian Film Com...,1977,feature-film,Break Of Day (1977),ba6eef73-d52d-4652-969a-ba27f7800879,c6a6eabc-c4fc-4160-a261-1c6904a14077
